# Queries de SQL aplicadas sobre la BBDD

El siguiente notebook busca responder las consignas del punto 2. Se ejecutarán las queries pertinentes, y se mostrarán en pantalla los resultados. 

*Observación: para correr adecuadamente este notebook, se requieren las credenciales de acceso a la base, las cuales no están publicadas en el repositorio público. A su vez, se deben instalar las bibliotecas que se importan a continuación*


In [1]:
# Importamos bibliotecas...
import pandas as pd
import sqlalchemy as sql
import os
from dotenv import load_dotenv

In [4]:
# Cargamos las credenciales de la base de datos
load_dotenv()
NAME=os.getenv("NAME")
PASSWORD=os.getenv("PASSWORD")
USER=os.getenv("USERNAME_AIVEN")
HOST=os.getenv("HOST")
PORT=os.getenv("PORT")
SSLMODE=os.getenv("SSL_MODE")

# Creamos la conexión
engine = sql.create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}?sslmode={SSLMODE}")

In [9]:
# Query de prueba
query = "SELECT * FROM pacientes LIMIT 5"
pd.read_sql(query, engine, index_col="id_paciente") 

,nombre,fecha_nacimiento,id_sexo,numero,calle,ciudad,edad
id_paciente,,,,,,,
1,Luciana Gómez,1991-07-12,2,121,Avenida Las Heras,Bs Aires,33
2,Ricardo López,1984-03-22,1,434,Calle San Juan,Rosario,40
3,Clara Fernández,1990-09-15,2,569,Calle San Juan,Rosario,34
4,Marcos Ramírez,1983-11-10,1,250,Avenida Pellegrini,Santa Fe,40
5,Julieta Rodríguez,1985-06-18,2,845,Calle Mitre,buenos aires,39


**1. Cuando se realizan consultas sobre la tabla paciente agrupando por ciudad los tiempos de respuesta son demasiado largos. 
Proponer mediante una query SQL una solución a este problema.**

In [16]:
query_1 = '''CREATE INDEX IF NOT EXISTS idx_ciudades 
             ON pacientes (ciudad);'''

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_1))
        print("Índice creado correctamente")
        
    except Exception as e:
        print(e)

Índice creado correctamente


In [18]:
query_2_1 = '''ALTER TABLE pacientes
                ADD COLUMN IF NOT EXISTS edad INT;'''
query_2_2 = '''UPDATE pacientes
                SET edad = EXTRACT(YEAR FROM AGE(CURRENT_DATE, fecha_nacimiento));'''
query_2_3 = '''CREATE INDEX IF NOT EXISTS idx_edades
                ON pacientes (edad);''' 

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_2_1))
        connection.execute(sql.text(query_2_2))
        connection.execute(sql.text(query_2_3))
        print("Columna e Índice creados correctamente")
        
    except Exception as e:
        print(e)

Columna e Índice creados correctamente


**3. La paciente, “Luciana Gómez”, ha cambiado de dirección. Antes vivía en “Avenida Las Heras 121” en “Buenos Aires”, pero ahora vive en “Calle Corrientes 500” en “Buenos Aires”. Actualizar la dirección de este paciente en la base de datos.**

In [21]:
query_3 =  '''UPDATE pacientes
                SET calle = 'Calle Corrientes', numero = 500
                WHERE nombre = 'Luciana Gómez';'''

with engine.connect() as connection:
    try:
        res = connection.execute(sql.text(query_3))
        print("Datos actualizados correctamente")
        print(f"{res.rowcount} fila(s) afectadas") 

    except Exception as e:
        print(e)

Datos actualizados correctamente
1 fila(s) afectadas


**4. Seleccionar el nombre y la matrícula de cada médico cuya especialidad sea identificada por el id 4.**

In [22]:
query_4 = '''SELECT m.nombre, m.matricula, e.nombre
            FROM medicos m JOIN especialidades e
            ON m.especialidad_id = e.id_especialidad
            WHERE e.id_especialidad = 4;'''
pd.read_sql(query_4, engine)

,nombre,matricula,nombre
0,Dra. Lucía Rodríguez,89012,Dermatología
1,Dr. Nicolás Gutiérrez,90123,Dermatología


**5. Puede pasar que haya inconsistencias en la forma en la que están escritos los nombres de las ciudades, ¿cómo se corrige esto? Agregar la query correspondiente.**

Para esta actividad, haremos uso de la biblioteca `pg_trgm`, la cual debemos importar a la base:

In [27]:
query_5_1 = "CREATE EXTENSION IF NOT EXISTS pg_trgm;"
query_5_2 = "SELECT * FROM pg_extension WHERE extname = 'pg_trgm';"

with engine.connect() as connection:
    try:
        connection.execute(sql.text(query_5_1))
        res = connection.execute(sql.text(query_5_2))
        print("Extensión creada correctamente")
        print(res.fetchall())
        connection.execute(sql.text('COMMIT;'))
        
    except Exception as e:
        print(e)

Extensión creada correctamente
[(16615, 'pg_trgm', 10, 2200, True, '1.6', None, None)]


In [32]:
ciudades = ['Buenos Aires', 'Rosario', 'Córdoba', 'Sante Fé', 'Mendoza']

with engine.connect() as connection:
    for ciudad in ciudades:
        query_5 = f'''UPDATE pacientes
                    SET ciudad = '{ciudad}'
                    WHERE similarity(ciudad, '{ciudad}') > 0.3;'''
        try:
            res = connection.execute(sql.text(query_5))
            print(f"Datos actualizados correctamente para {ciudad}")
            print(f"{res.rowcount} fila(s) afectadas\n")
        except Exception as e:
            print(f"Error en la ciudad {ciudad}")
            print(e)
    connection.execute(sql.text('COMMIT;'))


Datos actualizados correctamente para Buenos Aires
8 fila(s) afectadas

Datos actualizados correctamente para Rosario
4 fila(s) afectadas

Datos actualizados correctamente para Córdoba
6 fila(s) afectadas

Datos actualizados correctamente para Sante Fé
1 fila(s) afectadas

Datos actualizados correctamente para Mendoza
4 fila(s) afectadas

